In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap

# Add parent directory to sys.path in order to find common module
import sys
sys.path.insert(0,'../')

import common.datautils as datautils 

In [ ]:
ric_data = datautils.load_zip_results('RIC.results.zip', ['Benchmark','Flags', 'RunId', 'Success'])
ric_data.head()

In [ ]:
ce_average_data = datautils.load_ce_results('CE.results.zip')
ce_average_data.head()

In [ ]:
benchmarks = ric_data["Benchmark"].unique()
benchmarks = np.delete(benchmarks, 6) # Remove nab as something wrong with RIC results
print(benchmarks)

In [ ]:
def compare():
    overview = []

    for benchmark in benchmarks:
        benchmark_ce_data = ce_average_data[(ce_average_data["Benchmark"] == benchmark) & (ce_average_data["Flags"] != '-O3')]
        benchmark_ric_data = ric_data[(ric_data["Benchmark"] == benchmark) & (ric_data["Flags"] != '-O3')]

        min_ric_energy = benchmark_ric_data["Energy"].min()
        min_ric_time = benchmark_ric_data["Time"].min()
        total_ric_time = np.sum(benchmark_ric_data["Time"])

        min_ce_energy = benchmark_ce_data["Energy"].min()
        min_ce_time = benchmark_ce_data["Time"].min()
        total_ce_time = np.sum(benchmark_ce_data["Time"])

        overview.append(
            [
                
                np.round(min_ric_energy), 
                np.round(min_ce_energy),
                np.round(min_ric_time, 2),
                np.round(min_ce_time, 2),
                np.round(total_ric_time / 60 / 60, 2), # Time in hours,
                np.round(total_ce_time / 60 / 60, 2) # Time in hours
            ]
        )

    return pd.DataFrame(overview,
                        index=benchmarks,
                        columns=[
                            'Min RIC Energy',
                            'Min CE Energy',
                            'Min RIC Time',
                            'Min CE Time',
                            'Total RIC Time',
                            'Total CE Time'
                        ]
                       )
overview = compare()
overview

In [ ]:
print(f"Energy CE < RIC: {len(overview[overview['Min CE Energy'] < overview['Min RIC Energy']])}")
print(f"Time CE < RIC: {len(overview[overview['Min CE Time'] < overview['Min RIC Time']])}")
print(f"Total Time CE < RIC: {len(overview[overview['Total CE Time'] < overview['Total RIC Time']])}")

In [ ]:
def plot_energy_vs_time(benchmark, ce_data, ric_data):
    
    ric_benchmark_data = ric_data.loc[ric_data["Benchmark"] == benchmark]
    ce_benchmark_data = ce_data.loc[ce_data["Benchmark"] == benchmark]
    
    ric_o3_data = ric_benchmark_data.loc[ric_benchmark_data["Flags"] == "-O3"]
    ce_o3_data = ce_benchmark_data.loc[ce_benchmark_data["Flags"] == "-O3"]
    
    ric_non_o3_data = ric_benchmark_data.loc[ric_benchmark_data["Flags"] != "-O3"]
    ce_non_o3_data = ce_benchmark_data.loc[ce_benchmark_data["Flags"] != "-O3"]
    
    ric_X = ric_non_o3_data["Time"].values
    ric_Y = ric_non_o3_data["Energy"].values
    
    ce_X = ce_non_o3_data["Time"].values
    ce_Y = ce_non_o3_data["Energy"].values
    
    plt.figure()
    plt.scatter(ric_X,
                ric_Y,
                label='RIC Configurations',
                c='C1')

    
    plt.scatter(ce_X,
                ce_Y,
                label='CE Configurations',
                c='C0')
    
    
    plt.scatter(ce_o3_data["Time"].values,
                ce_o3_data["Energy"].values,
                c='r',
                label='-O3')
#     plt.scatter(ric_o3_data["Time"].values,
#                 ric_o3_data["Energy"].values,
#                 c='g',
#                 label='-O3')

    
    
    plt.title('Energy vs Time results for CE and RIC on ' + benchmark)
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (J)')
    plt.grid()

In [ ]:
for benchmark in benchmarks:
    plot_energy_vs_time(benchmark, ce_average_data, ric_data)